# 性能分析 Performance Analysis

本节我们将通过查看TopDown计数器、查看Cache MPKI数据、SPEC06算分三个部分，介绍基本的性能分析流程。

In this section, we introduce the basic performance analysis workflow through three aspects: checking TopDown counters, checking cache MPKI data, and calculating SPEC06 scores.

## 查看 TopDown 计数器 Check TopDown counters

我们使用 gem5_data_proc 仓库下的脚本来处理上节运行 GEM5 得到的原始数据，获取 top-down 计数器。

We use the scripts in the gem5_data_proc repository to process the raw data generated by GEM5 in the previous section and obtain the TopDown counters.

In [ ]:
%%bash
source ../env.sh

python3 ${GEM5_DATA_PROC_HOME}/batch.py \
    -s ${WORK_DIR}/04-gem5/02-first-run \
    -t --topdown-raw

脚本运行完成后，我们将看到上一节运行 Hello World 程序后收集到的 Top-Down 性能计数器。

结果会展示整体的 IPC、Commit_Squash 等关键性能指标。

After the script finishes running, we will see the Top-Down performance counters collected from the Hello World execution in the previous section.

The results will display key performance metrics such as IPC, Commit_Squash.

## 查看 Cache MPKI数据 Check Cache MPKI data

类似地，我们也可以查看 Cache MPKI 数据。由于上一节的 hello-world 程序缺乏代表性，这里我们改用预先运行的 SPEC06 切片结果来进行更具参考价值的分析。


Similarly, we can further examine the Cache MPKI data. As the hello-world program used in the previous section lacks representativeness, we instead use pre-executed SPEC06 slices to conduct a more insightful analysis.

In [ ]:
%%bash
source ../env.sh

mkdir -p ${WORK_DIR}/04-gem5/03-perf
cd ${WORK_DIR}/04-gem5/03-perf

export PYTHONPATH=${GEM5_DATA_PROC_HOME}
ulimit -n 65535

python3 ${GEM5_DATA_PROC_HOME}/batch.py \
    -s $(get_asset gem5-spec-topdown-result) \
    -o gem5-cache-example.csv \
    --cache \
    > /dev/null

mkdir -p results

python3 ${GEM5_DATA_PROC_HOME}/simpoint_cpt/compute_weighted.py \
    -r gem5-cache-example.csv \
    -j $(get_asset gem5-spec-topdown-result/cluster-0-0.json) \
    -o weighted.csv \
    | tail -n 33

这一段程序会调用 Top-Down 脚本，对 XS-GEM5 运行 SPEC06 的性能计数器进行分析。

我们可以观察到 Cache 的 MPKI 等关键指标，从而便于后续开展更深入的性能分析。



This program uses the Top-Down script to analyze the performance counters of XS-GEM5 running SPEC06.

We can observe key metrics such as Cache MPKI, which makes it easier to conduct further performance analysis.


## SPEC06 算分 Calculate SPEC06 scores

我们还可以根据切片的权重和运行结果**估算**运行完整 SPEC06 的分数，评估整体的性能。

We can also estimate the overall performance by calculating the SPEC06 score based on the weights of the slices and their execution results.

In [ ]:
%%bash
source ../env.sh

mkdir -p ${WORK_DIR}/04-gem5/03-perf
cd ${WORK_DIR}/04-gem5/03-perf

export PYTHONPATH=${GEM5_DATA_PROC_HOME}
ulimit -n 65535

python3 ${GEM5_DATA_PROC_HOME}/batch.py \
    -s $(get_asset gem5-spec-topdown-result) \
    -o gem5-score-example.csv \
    > /dev/null

mkdir -p results

python3 ${GEM5_DATA_PROC_HOME}/simpoint_cpt/compute_weighted.py \
    -r gem5-score-example.csv \
    -j $(get_asset gem5-spec-topdown-result/cluster-0-0.json) \
    --score score.csv \
    | tail -n 41

我们通过这个 Bash 脚本运行 SPEC06 的算分脚本来获得性能分数。

在当前的默认配置下，XS-GEM5 的整数分数为 45.36，浮点分数为 49.64，对应频率为 3 GHz。


We use this Bash script to run the SPEC06 scoring script and obtain the performance scores.

Under the current default configuration, XS-GEM5 achieves an int score of 45.36 and a fp score of 49.64 at 3 GHz.